In [1]:
import os
os.chdir('/home/yash/Desktop/Decision-Flip-Experiments')
from scipy.misc import imread
import matplotlib.patches as mpatches
from models import *
from plotter import *
from saveloader import *
from fgsm_cifar import fgsm
from fgsm_cifar_wrt_class_flipstop import fgsm_wrt_class
from helper import *
from helper import _to_categorical
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
sd = 'shape_dict'





def make_data(X_test, X_train, sess, n, env, n_classes=10):
    X_test_sub = X_test[:n]
    X_train_sub = X_train[:n]
    y_train_sub = sess.run(env.ybar, feed_dict={env.x: X_train_sub, env.training: False})
    y_train_sub = _to_categorical(np.argmax(y_train_sub, axis=1), n_classes)
    y_test_sub = sess.run(env.ybar, feed_dict={env.x: X_test_sub, env.training: False})
    y_test_sub = _to_categorical(np.argmax(y_test_sub, axis=1), n_classes)

    return X_test_sub, y_test_sub, X_train_sub, y_train_sub

def plot_all_data_graph(method, epochs, n):
    for from_cls in range(2):
        for to_cls in range(n_classes):
            if (from_cls != to_cls):
                l2_test, l2_train, l2_random, l2_random_normal = restore_flip(method, epochs, n, from_cls, to_cls)
                #There might be a prob. here! since lens are diff, solved it inside the func
                plot_data_graph_without_random(l2_test, l2_train, l2_random, l2_random_normal, n, from_cls, to_cls)
                plot_hists_without_random(l2_test, l2_train, n, from_cls, to_cls)

def random_normal_func(X, n, save, lr, lrn):
    X = X.reshape(-1, img_rows * img_cols * img_chas)
    mean, std = np.mean(X, axis=0), np.std(X, axis=0)
    randomX = np.zeros([n, X[0].size])
    for i in range(X[0].size):
        randomX[:, i] = np.random.normal(mean[i], std[i], n)
    randomX = randomX.reshape(-1, img_rows, img_cols, img_chas)
    X_random_normal = randomX
    ans = sess.run(env.ybar, feed_dict={env.x: randomX, env.training: False})
    y_random_normal = _to_categorical(np.argmax(ans, axis=1), n_classes)
    X_random = np.random.rand(n, img_rows, img_cols, img_chas)
    y_random = sess.run(env.ybar, feed_dict={env.x: X_random, env.training: False})
    y_random = _to_categorical(np.argmax(y_random, axis=1), n_classes)

    if (save):
        save_as_txt(get_flip_path(lr), X_random)
        save_as_txt(get_flip_path(lrn), X_random_normal)

    return X_random, y_random, X_random_normal, y_random_normal

def run_flip(method, epochs, n, cls=-1):
    save_obj({},sd)
    test_label = make_label("test", method, epochs, n, False)
    train_label = make_label("train", method, epochs, n, False)
    random_label = make_label("random", method, epochs, n, False)
    random_normal_label = make_label("random_normal", method, epochs, n, False)
    data_label_random, data_label_random_normal = make_label("_", method, epochs, n, True)

    X_test_sub, y_test_sub, X_train_sub, y_train_sub = make_data(X_test, X_train, sess, n, env)
    X_random, y_random, X_random_normal, y_random_normal = random_normal_func(X_train, n, True,
                                                                              data_label_random,
                                                                              data_label_random_normal)
    if (cls < -2 or cls > 9):
        print("Invalid Params")
        return

    if (method == 1):
        X_flip_test = create_adv(X_test_sub, y_test_sub, test_label)
        X_flip_train = create_adv(X_train_sub, y_train_sub, train_label)
        X_flip_random = create_adv(X_random, y_random, random_label)
        X_flip_random_normal = create_adv(X_random_normal, y_random_normal, random_normal_label)

    if (method == 2):
        X_flip_per_class_test = create_adv_wrt_class(X_test_sub, y_test_sub, test_label)
        X_flip_per_class_train = create_adv_wrt_class(X_train_sub, y_train_sub, train_label)
        X_flip_per_class_random = create_adv_wrt_class(X_random, y_random, random_label)
        X_flip_per_class_random_normal = create_adv_wrt_class(X_random_normal, y_random_normal, random_normal_label)

        _, X_flip_test = give_m2_ans(X_test_sub, X_flip_per_class_test, cls)
        _, X_flip_train = give_m2_ans(X_train_sub, X_flip_per_class_train, cls)
        _, X_flip_random = give_m2_ans(X_random, X_flip_per_class_random, cls)
        _, X_flip_random_normal = give_m2_ans(X_random_normal, X_flip_per_class_random_normal, cls)

    #     a = _predict(X_flip_test, env)
    #     print(np.argmax(a,axis=1))

    l2_test = find_l2(X_flip_test, X_test_sub)
    l2_train = find_l2(X_flip_train, X_train_sub)
    l2_random = find_l2(X_flip_random, X_random)
    l2_random_normal = find_l2(X_flip_random_normal, X_random_normal)

    return l2_test, l2_train, l2_random, l2_random_normal

def restore_random_data(lr, lrn):
    Xr = load_from_txt(get_flip_path(lr))
    Xrn = load_from_txt(get_flip_path(lrn))
    y_random_normal = sess.run(env.ybar, feed_dict={env.x: Xrn, env.training: False})
    y_random_normal = _to_categorical(np.argmax(y_random_normal, axis=1), n_classes)
    y_random = sess.run(env.ybar, feed_dict={env.x: Xr, env.training: False})
    y_random = _to_categorical(np.argmax(y_random, axis=1), n_classes)

    return Xr, y_random, Xrn, y_random_normal

def restore_flip(method, epochs, n, from_cls=-1, to_cls=-1):
    test_label = make_label("test", method, epochs, n, False)
    train_label = make_label("train", method, epochs, n, False)
    random_label = make_label("random", method, epochs, n, False)
    random_normal_label = make_label("random_normal", method, epochs, n, False)
    data_label_random, data_label_random_normal = make_label("_", method, epochs, n, True)

    X_test_sub, y_test_sub, X_train_sub, y_train_sub = make_data(X_test, X_train, sess, n, env)
    X_random, y_random, X_random_normal, y_random_normal = restore_random_data(data_label_random,
                                                                               data_label_random_normal)

    if (method == 1):
        X_flip_test = load_from_txt(get_flip_path(test_label))
        X_flip_train = load_from_txt(get_flip_path(train_label))
        X_flip_random = load_from_txt(get_flip_path(random_label))
        X_flip_random_normal = load_from_txt(get_flip_path(random_normal_label))

    if (method == 2):
        X_flip_per_class_test = load_from_txt(get_flip_path(test_label))
        X_flip_per_class_train = load_from_txt(get_flip_path(train_label))
        X_flip_per_class_random = load_from_txt(get_flip_path(random_label))
        X_flip_per_class_random_normal = load_from_txt(get_flip_path(random_normal_label))

        if (from_cls != -1):
            print('From Class ' + str(from_cls) + '\n')
            X_test_sub, y_test_sub, X_flip_per_class_test = get_class(X_test_sub, y_test_sub,
                                                                      X_flip_per_class_test, from_cls)
            X_train_sub, y_train_sub, X_flip_per_class_train = get_class(X_train_sub, y_train_sub,
                                                                         X_flip_per_class_train, from_cls)
            X_random, y_random, X_flip_per_class_random = get_class(X_random, y_random,
                                                                    X_flip_per_class_random, from_cls)
            X_random_normal, y_random_normal, X_flip_per_class_random_normal = get_class(
                X_random_normal, y_random_normal, X_flip_per_class_random_normal, from_cls)
            print('Test Data:' + str(y_test_sub.shape[0]))
            print('Train Data: ' + str(y_train_sub.shape[0]))
            print('Random Data: ' + str(y_random.shape[0]))
            print('Random Normal Data: ' + str(y_random_normal.shape[0]))

        _, X_flip_test = give_m2_ans(X_test_sub, X_flip_per_class_test, to_cls)
        _, X_flip_train = give_m2_ans(X_train_sub, X_flip_per_class_train, to_cls)
        _, X_flip_random = give_m2_ans(X_random, X_flip_per_class_random, to_cls)
        _, X_flip_random_normal = give_m2_ans(X_random_normal, X_flip_per_class_random_normal, to_cls)

    l2_test = find_l2(X_flip_test, X_test_sub)
    l2_train = find_l2(X_flip_train, X_train_sub)
    l2_random = find_l2(X_flip_random, X_random)
    l2_random_normal = find_l2(X_flip_random_normal, X_random_normal)

    return l2_test, l2_train, l2_random, l2_random_normal

X_train, y_train, X_test, y_test, X_valid, y_valid = load_data(os)


class Dummy:
    pass

env = Dummy()


/home/yash/Desktop/Decision-Flip-Experiments/plotter.py:4: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/ev


Loading CIFAR10

Loading pre-Shuffled training data


In [2]:
# We need a scope since the inference graph will be reused later
with tf.variable_scope('model'):
    env.x = tf.placeholder(tf.float32, (None, img_rows, img_cols,
                                        img_chas), name='x')
    env.y = tf.placeholder(tf.float32, (None, n_classes), name='y')
    env.training = tf.placeholder(bool, (), name='mode')

    env.ybar, logits = model(env.x, logits=True,
                             training=env.training)

    z = tf.argmax(env.y, axis=1)
    zbar = tf.argmax(env.ybar, axis=1)
    env.count = tf.cast(tf.equal(z, zbar), tf.float32)
    env.acc = tf.reduce_mean(env.count, name='acc')

    xent = tf.nn.softmax_cross_entropy_with_logits(labels=env.y,
                                                   logits=logits)
    env.loss = tf.reduce_mean(xent, name='loss')

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.control_dependencies(extra_update_ops):
    env.optim = tf.train.AdamOptimizer(beta1=0.9, beta2=0.999, epsilon=1e-08,).minimize(env.loss)

with tf.variable_scope('model', reuse=True):
    for i in range(n_classes):
        if (i == 0):
            env.x_adv_wrt_class = (fgsm_wrt_class(model, env.x, i, step_size=.05, bbox_semi_side=10))
        else:
            x = (fgsm_wrt_class(model, env.x, i, step_size=.05, bbox_semi_side=10))
            env.x_adv_wrt_class = tf.concat([env.x_adv_wrt_class, x], axis=0)
    env.x_adv, env.all_flipped = fgsm(model, env.x, step_size=.05, bbox_semi_side=10)  # epochs is redundant now!

In [3]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

In [4]:
def save_model(label):
    saver = tf.train.Saver()
    saver.save(sess,  './models/cifar/' + label)
    
def restore_model(label):
    saver = tf.train.Saver()
    saver.restore(sess, './models/cifar/' + label)

In [5]:
def _evaluate(X_data, y_data, env):
    print('\nEvaluating')
    n_sample = X_data.shape[0]
    batch_size = 128
    n_batch = int(np.ceil(n_sample/batch_size))
    loss, acc = 0, 0
    ns = 0
    for ind in range(n_batch):
        print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
        start = ind*batch_size
        end = min(n_sample, start+batch_size)
        batch_loss, batch_count, batch_acc = sess.run(
            [env.loss, env.count, env.acc],
            feed_dict={env.x: X_data[start:end],
                       env.y: y_data[start:end],
                       env.training: False})
        loss += batch_loss*batch_size
        ns+=batch_size
        acc += batch_acc*batch_size
    loss /= ns
    acc /= ns
    print(' loss: {0:.4f} acc: {1:.4f}'.format(loss, acc))
    return loss, acc

In [6]:
def _predict(X_data, env):
    print('\nPredicting')
    n_sample = X_data.shape[0]
    batch_size = 128
    n_batch = int(np.ceil(n_sample/batch_size))
    yval = np.empty((X_data.shape[0], n_classes))
    for ind in range(n_batch):
        print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
        start = ind*batch_size
        end = min(n_sample, start+batch_size)
        batch_y = sess.run(env.ybar, feed_dict={
            env.x: X_data[start:end], env.training: False})
        yval[start:end] = batch_y
    return yval

def train(label):
    print('\nTraining')
    n_sample = X_train.shape[0]
    batch_size = 128
    n_batch = int(np.ceil(n_sample/batch_size))
    n_epoch = 50
    for epoch in range(n_epoch):
        print('Epoch {0}/{1}'.format(epoch+1, n_epoch))
        for ind in range(n_batch):
            print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
            start = ind*batch_size
            end = min(n_sample, start+batch_size)
            sess.run(env.optim, feed_dict={env.x: X_train[start:end],
                                           env.y: y_train[start:end],
                                           env.training: True})
        if(epoch%5 == 0):
            model_label = label+ '{0}'.format(epoch)
            print("saving model " + model_label)
            save_model(model_label)
            
    save_model(label)

def train_again(X, y, epochs):
    #Not making batches, do that if size > 128
    for i in range(X.shape[0]):
        for e in range(epochs):
            sess.run(env.optim, feed_dict={env.x: [X[i]],
                                           env.y: [y[i]],
                                           env.training: True})
            
    
    

In [7]:
def create_adv_wrt_class(X, Y, label = None):
    print('\nCrafting adversarial')
    n_sample = X.shape[0]
    pred = np.argmax(Y,axis=1)
    batch_size = 1
    n_batch = int(np.ceil(n_sample/batch_size))
    n_epoch = 20
    x_adv_shape = list(X.shape)[1:]
    x_adv_shape = np.append(np.append(n_sample,n_classes),x_adv_shape)
    X_adv = np.empty(x_adv_shape)
    for ind in range(n_batch):
        print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
        start = ind*batch_size
        end = min(n_sample, start+batch_size)
        tmp = sess.run(env.x_adv_wrt_class, feed_dict={env.x: X[start:end],
                                             env.y: Y[start:end],
                                             env.training: False})
        tmp[pred[start]] = X[start]
        X_adv[start:end] = tmp
    if(label != None):
        print('\nSaving adversarial')
        os.makedirs('data', exist_ok=True)
        save_as_txt(get_flip_path(label), X_adv)
    return X_adv

In [8]:
def create_adv(X, Y, label):
    print('\nCrafting adversarial')
    n_sample = X.shape[0]
    batch_size = 1
    n_batch = int(np.ceil(n_sample/batch_size))
    n_epoch = 20
    X_adv = np.empty_like(X)
    for ind in range(n_batch):
        print(' batch {0}/{1}'.format(ind+1, n_batch), end='\r')
        start = ind*batch_size
        end = min(n_sample, start+batch_size)
        tmp, all_flipped = sess.run([env.x_adv, env.all_flipped], feed_dict={env.x: X[start:end],
                                             env.y: Y[start:end],
                                             env.training: False})
        X_adv[start:end] = tmp
    print('\nSaving adversarial')
    os.makedirs('data', exist_ok=True)
    save_as_txt(get_flip_path(label), X_adv)
    return X_adv

In [9]:
method = 2
n = 1000
epochs = 200
label="cifar_with_cnn"
cls = -1

# train(label)
restore_model(label + str(epochs))

X = X_train[:50]
y = y_train[:50]

lbl = "temp_exp_flipstop"
Xflip = create_adv_wrt_class(X,y,lbl)
Xf = load_from_txt(get_flip_path(lbl))w
# print(Xflip == Xf)

INFO:tensorflow:Restoring parameters from ./models/cifar/cifar_with_cnn200

Crafting adversarial
 batch 50/50
Saving adversarial


In [14]:
for i in range (len(Xf)):
    pred = _predict(Xf[i],env)
    a = (np.argmax(pred, axis = 1))
    print(a)
    c = (np.count_nonzero(a-[0,1,2,3,4,5,6,7,8,9]))
    print('No.of multiple hits: ' + str(c))


Predicting
[8 1 0 9 8 8 8 8 8 9]
7

Predicting
[0 1 2 2 4 5 6 4 8 4]
3

Predicting
[6 6 6 6 4 5 6 7 6 6]
6

Predicting
[7 7 2 3 4 7 6 7 4 9]
4

Predicting
[4 4 2 3 4 6 6 7 4 4]
5

Predicting
[0 1 2 2 2 5 2 7 8 9]
3

Predicting
[0 1 2 3 4 1 6 1 0 9]
3

Predicting
[4 1 3 3 4 5 3 4 4 4]
6

Predicting
[3 4 4 3 4 5 6 7 3 3]
5

Predicting
[3 1 2 6 4 3 6 3 8 9]
4

Predicting
[0 0 2 3 4 5 0 7 0 0]
4

Predicting
[0 1 2 2 2 2 2 2 8 9]
5

Predicting
[3 2 2 3 4 5 6 7 3 9]
3

Predicting
[0 1 0 3 8 8 8 7 8 9]
4

Predicting
[0 1 2 3 2 5 6 5 2 6]
4

Predicting
[8 1 4 3 4 5 4 8 8 9]
4

Predicting
[0 1 0 3 0 5 3 3 8 0]
5

Predicting
[0 7 2 3 4 5 6 7 3 3]
3

Predicting
[4 1 4 4 4 5 4 4 8 9]
5

Predicting
[0 9 2 5 4 5 6 9 8 9]
3

Predicting
[0 1 5 3 5 5 5 7 8 9]
3

Predicting
[4 4 2 3 4 3 6 4 4 9]
5

Predicting
[6 1 6 6 6 6 6 6 8 6]
7

Predicting
[0 9 2 9 9 9 6 5 1 9]
6

Predicting
[3 0 3 3 4 5 6 7 3 3]
5

Predicting
[0 1 9 9 9 8 9 9 8 9]
6

Predicting
[5 1 4 5 4 5 6 7 8 9]
3

Predicting
[0 1 2 3 4 7 0 7

In [13]:
lbl = "temp_exp_flipstop200e"

save_as_txt(get_flip_path(lbl), Xflip)